# Overview of the CrystFEL-based processing

This notebook comprises a workflow using CrystFEL-based tools combined with original scripts. Below is a step-by-step guide:

1. **Run Indexamajig** (`gandalf_iterator`)  
   - Perform peakfinding, indexing, and integration for each HDF5 file in the specified folder, varying beam center coordinates on a grid withing a given radius.

2. **Evaluate IQM** (`automate_evaluation`)  
   - Parse stream files for indexing quality metrics (IQMs), apply weights, and identify the best results.

3. **Merge** (`merge`)  
   - Merge the best result stream file to refine cell parameters and symmetry.

4. **Convert**  
   - **to `.hkl`** for ShelX (`convert_hkl_crystfel_to_shelx`).  
   - **to `.mtz`** for downstream crystallographic tools (`convert_hkl_to_mtz`).

Please ensure that preprocessing (peak finding, center finding and center refinement) has been done and that you have the required packages and environment set up (CrystFEL, Python packages, etc.) before proceeding.


# Run Indexamajig with options for peakfinding, indexing and integration

In [ ]:
from gandalf_radial_iterator import gandalf_iterator

geomfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTAsim.geom"       # .geom file
cellfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTA.cell"          # .cell file

input_path =   "/Users/xiaodong/Desktop/simulations/LTA/simulation-24"      # .h5 folder will also be output folder

output_file_base = "LTA"    # output files will be named output_file_base_xcoord_ycoord.h5

num_threads = 8             # number of CPU threads to use
x, y = 512.5, 512.5         # initial beam center from where iterations will start

"""Define the grid and maximum radius for iterations.
As example max_radius = 1, step = 0.2 will give 81 iterations.
Iterations will start at the center and move radially outwards.
"""
max_radius = 1              # maximum radius in pixels
step = 0.2                  # grid granularity in pixels

extra_flags=[
# PEAKFINDING
"--no-revalidate",
"--no-half-pixel-shift",
"--peaks=cxi", 
"--min-peaks=15",
# INDEXING
"--indexing=xgandalf",
"--tolerance=10,10,10,5",
"--no-refine",
"--xgandalf-sampling-pitch=5",
"--xgandalf-grad-desc-iterations=1",
"--xgandalf-tolerance=0.02",
# INTEGRATION
"--integration=rings",
"--int-radius=4,5,9",
"--fix-profile-radius=70000000",
# OUTPUT
"--no-non-hits-in-stream",
]

"""Examples of extra flags(see crystfel documentation https://www.desy.de/~twhite/crystfel/manual-indexamajig.html):"""

""" Basic options
"--highres=n",
"--no-image-data",
"""

""" Peakfinding
"--peaks=cxi",
"--peak-radius=inner,middle,outer",
"--min-peaks=n",
"--median-filter=n",
"--filter-noise",
"--no-revalidate",
"--no-half-pixel-shift",

"--peaks=peakfinder9",
"--min-snr=1",
"--min-snr-peak-pix=6",
"--min-snr-biggest-pix=1",
"--min-sig=9",
"--min-peak-over-neighbour=5",
"--local-bg-radius=5",

"--peaks=peakfinder8",
"--threshold=45",
"--min-snr=3",
"--min-pix-count=3",
"--max-pix-count=500",
"--local-bg-radius=9",
"--min-res=30",
"--max-res=500",
"""

""" Indexing
"--indexing=xgandalf",

"--tolerance=tol"
"--no-check-cell",
"--no-check-peaks",
"--multi",
"--no-retry",
"--no-refine",

"--xgandalf-sampling-pitch=n"
"--xgandalf-grad-desc-iterations=n"
"--xgandalf-tolerance=n"
"--xgandalf-no-deviation-from-provided-cell"
"--xgandalf-max-lattice-vector-length=n"
"--xgandalf-min-lattice-vector-length=n"
"--xgandalf-max-peaks=n"
"--xgandalf-fast-execution"
"""

""" Integration
"--fix-profile-radius=n",
"--fix-divergence=n",
"--integration=rings",
"--int-radius=4,5,10",
"--push-res=n",
"--overpredict",
"--cell-parameters-only",
"""

""" Output
"--no-non-hits-in-stream",
"--no-peaks-in-stream",
"--no-refls-in-stream",
"--serial-offset
"""

gandalf_iterator(x, y, geomfile_path, cellfile_path, input_path, output_file_base, num_threads, max_radius=max_radius, step=step, extra_flags=extra_flags)


# Evaluate the IQM with chosen weights for all frames across all index results

In [2]:
from automate_evaluation import automate_evaluation

# Enter folder with stream file results from indexamajig. 
# Note that ALL stream files in the folder will be processed.
stream_file_folder = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2" 

weights_list = [
    (1, 1, 1, 1, 1, 1)
]

"""
Each weight corresponds to one of the six metrics used in calculating the combined IQM value.
The combined IQM is computed by first normalizing each metric across all stream files, then 
multiplying each normalized metric by its assigned weight, and finally summing the results.
The order (or keys) of the weights must match the following metrics:

- 'weighted_rmsd'
- 'fraction_outliers'
- 'length_deviation'
- 'angle_deviation'
- 'peak_ratio'
- 'percentage_indexed'

Multiple weight combinations can be specified if needed.
"""

automate_evaluation(stream_file_folder, weights_list, indexing_tolerance=2)

Evaluating multiple stream files with weights: (1, 1, 1, 1, 1, 1)


Processing chunks in LTA_-512.5_-512.7.stream: 100%|██████████| 100/100 [00:07<00:00, 13.72chunk/s]


Combined metrics CSV written to /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/metric_values_IQM_1_1_1_1_1_1.csv
Best results stream file written to /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1.stream


In [2]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from interactive_iqm import (
    read_metric_csv, 
    select_best_results_by_event, 
    get_metric_ranges,
    filter_rows,
    write_filtered_stream
)


# 1) Set the paths to your CSV and your existing "best results" .stream
CSV_PATH = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/metric_values_IQM_1_1_1_1_1_1.csv"
BEST_STREAM_PATH = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1.stream"
FILTERED_STREAM_PATH = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_filtered.stream"

# 2) Read the CSV (grouped by event string)
grouped_data = read_metric_csv(CSV_PATH, group_by_event=True)

# 3) If your CSV has multiple rows per event_number, pick "best" by combined_metric
best_rows = select_best_results_by_event(grouped_data, sort_metric='combined_metric')

# 4) Find the min/max range of each metric in best_rows
metrics_in_order = [
    'combined_metric', 'weighted_rmsd', 'fraction_outliers',
    'length_deviation', 'angle_deviation', 'peak_ratio',
    'percentage_unindexed'
]
ranges_dict = get_metric_ranges(best_rows, metrics=metrics_in_order)

# 5) Create sliders automatically based on actual data range
metric_sliders = {}

def create_slider(metric_name, min_val, max_val):
    # We'll do a ≤ filter, so let's default the slider to the max for "include all"
    default_val = max_val
    step = (max_val - min_val) / 100.0 if max_val != min_val else 0.01
    
    slider = widgets.FloatSlider(
        value=default_val,
        min=min_val,
        max=max_val,
        step=step,
        description=f'{metric_name} ≤'
    )
    return slider

for metric in metrics_in_order:
    mn, mx = ranges_dict[metric]
    metric_sliders[metric] = create_slider(metric, mn, mx)

# 6) Create a button to run the filtering & show histograms
filter_button = widgets.Button(description="Apply Threshold & Show Histograms")
output_area = widgets.Output()

def make_histograms(rows, metrics=None, bins=20):
    """Simple function to plot separate histograms (one figure per metric)."""
    if metrics is None:
        metrics = metrics_in_order
    for m in metrics:
        values = [r[m] for r in rows]
        plt.figure()
        plt.hist(values, bins=bins)
        plt.title(f'Histogram of {m}')
        plt.xlabel(m)
        plt.ylabel('Count')
        plt.show()

def on_filter_clicked(_):
    with output_area:
        output_area.clear_output()

        # Build threshold dict from the current slider values
        thresholds = {m: metric_sliders[m].value for m in metrics_in_order}

        # Filter the best_rows
        filtered = filter_rows(best_rows, thresholds)

        print(f"Filtering... {len(best_rows)} events -> {len(filtered)} pass thresholds.\n")
        
        if filtered:
            make_histograms(filtered, metrics_in_order)
        else:
            print("No events passed the thresholds, skipping histograms.")

        # Write a new .stream containing only filtered events
        write_filtered_stream(filtered, BEST_STREAM_PATH, FILTERED_STREAM_PATH)

filter_button.on_click(on_filter_clicked)

# 7) Display everything
all_widgets = list(metric_sliders.values()) + [filter_button, output_area]
display(*all_widgets)


FloatSlider(value=1.528445243520403, description='combined_metric ≤', max=1.528445243520403, min=-7.4551270789…

FloatSlider(value=-0.08409381599086142, description='weighted_rmsd ≤', max=-0.08409381599086142, min=-0.390105…

FloatSlider(value=0.8743852735883026, description='fraction_outliers ≤', max=0.8743852735883026, min=-1.674371…

FloatSlider(value=2.066147638068251, description='length_deviation ≤', max=2.066147638068251, min=-2.193098975…

FloatSlider(value=1.4546725967396552, description='angle_deviation ≤', max=1.4546725967396552, min=-2.30275985…

FloatSlider(value=1.0819393459096938, description='peak_ratio ≤', max=1.0819393459096938, min=-2.7932586938107…

FloatSlider(value=0.007790796995297262, description='percentage_unindexed ≤', max=0.007790796995297262, min=-0…

Button(description='Apply Threshold & Show Histograms', style=ButtonStyle())

Output()

# Merge the best results stream file

In [3]:
from merge import merge

stream_file = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1.stream"
pointgroup = "m-3m"
num_threads = 24
iterations = 5

output_dir = merge(
    stream_file,
    pointgroup=pointgroup,
    num_threads=num_threads,
    iterations=iterations,
)

if output_dir is not None:
    print("Merging done. Results are in:", output_dir)

Running partialator for stream file: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1.stream


Partialator Progress:   0%|          | 0/7 [00:00<?, ?Residual/s]

Partialator Progress: 100%|██████████| 7/7 [00:01<00:00,  6.78Residual/s]

Partialator completed for stream file: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1.stream
Merging done. Results are in: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter


# Convert to shelx compatible .hkl

In [2]:
from convert_hkl_crystfel_to_shelx import convert_hkl_crystfel_to_shelx 
output_dir = "/Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter"
convert_hkl_crystfel_to_shelx(output_dir)

[INFO] Converting crystfel.hkl to shelx.hkl in directory: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter
[INFO] Conversion to shelx.hkl completed successfully in: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter/shelx


# Convert to mtz

In [3]:
from convert_hkl_to_mtz import convert_hkl_to_mtz
cellfile_path = "/Users/xiaodong/Desktop/simulations/LTA/LTA.cell"  # If defined above comment out this line
convert_hkl_to_mtz(output_dir, cellfile_path=cellfile_path)

[INFO] Converting crystfel.hkl to output.mtz in directory: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter
[INFO] Conversion to output.mtz completed successfully in: /Users/xiaodong/Desktop/simulations/LTA/simulation-24/xgandalf_iterations_max_radius_1_step_0.2/merged_IQM_1_1_1_1_1_1_merge_5_iter
